## Python notebook for SOM

### Diego Polo 3/5/2020
Inpiration taken from
http://www.ai-junkie.com/ann/som/som1.html
https://visualstudiomagazine.com/articles/2019/01/01/self-organizing-maps-python.aspx
http://blog.yhat.com/posts/self-organizing-maps-2.html

In [46]:
## Dataset

#    Data will be a collection of random colours, so first we’ll artificially create a dataset of 100.
#    Each colour is a 3D vector representing R, G and B values

import numpy as np
raw_data = np.random.randint(0,255, size=(3,100))
print(raw_data.shape)

(3, 100)


In [47]:
## Setup

network_dimensions = np.array([5, 5])
n_iterations = 2000
init_learning_rate = 0.01

m = raw_data.shape[0]
n = raw_data.shape[1]

# weight matrix (i.e. the SOM) needs to be one m-dimensional vector for each neuron in the SOM
# in this case, a matrix of 5x5 but and each element is an array of 3 dimensions (m)
net = np.random.random((network_dimensions[0], network_dimensions[1], m))

# initial neighbourhood radius
init_radius = max(network_dimensions[0], network_dimensions[1]) / 2

# radius decay parameter
time_constant = n_iterations / np.log(init_radius)

In [48]:
## Normalisation

# Another detail to discuss at this point is whether or not we normalise our dataset.
# First of all, SOMs train faster (and “better”) if all our values are between 0 and 1.
# This is to avoid one of our dimensions “dominating” the others in the learning process.

# we want to keep a copy of the raw data for later
data = raw_data

normalise_data = True
normalise_by_column = True # no need as all colums have same range

# check if data needs to be normalised
if normalise_data:
    if normalise_by_column:
        # normalise along each column
        col_maxes = raw_data.max(axis=0)
        data =   raw_data / col_maxes[np.newaxis, :]
    else:
        # normalise entire dataset
        data = raw_data / data.max()


In [49]:
## Learning

# decay the SOM parameters
# We want to decay the learning rate over time to let the SOM “settle” on a solution.

# What we also decay is the neighbourhood radius, which defines how far we search for 2D neighbours when updating vectors in the SOM. 
# We want to gradually reduce this over time, like the learning rate.

# The functions to decay the radius and learning rate use exponential decay: σt = σ0·exp(-t/λ)
# Where λ is the time constant (which controls the decay) and σ is the value at various times t.

def decay_radius(initial_radius, iteration, lambdaa):
    return initial_radius * np.exp(-iteration / lambdaa)

def decay_learning_rate(initial_learning_rate, iteration, n_iterations):
    return initial_learning_rate * np.exp(-iteration / n_iterations)



# Find the neuron in the SOM whose associated 3D vector is closest to our chosen 3D colour vector.
# At each step, this is called the Best Matching Unit (BMU)
#     For that to work we need a function to find the BMU. It need to iterate through each neuron in the SOM, measure its Euclidean distance to our input vector and return the one that’s closest. Note the implementation trick of not actually measuring Euclidean distance, but the squared Euclidean distance, thereby avoiding an expensive square root computation.

def find_bmu(t, net):
    """
    Find the best matching unit for a given vector, t, in the SOM
    Returns: a (bmu, bmu_idx) tuple where bmu is the high-dimensional BMU
                and bmu_idx is the index of this vector in the SOM
    """
    min_dist = np.Infinity
    for x in range(net.shape[0]):
        for y in range(net.shape[1]):
            w = net[x, y, :]
            # calculate kinda Euclidean distance
            dist = np.sum((w - t)**2)
            if dist < min_dist:
                min_dist = dist
                x_min = x
                y_min = y
    # get the vector corresponding to x_min and y_min
    return (net[x, y, :], x_min, y_min)

# Move the BMU’s 3D weight vector closer to the input vector in 3D space
# The formula to update the BMU’s 3D vector is: Wt+1 = Wt + L(Vt-Wt)
# That is to say, the new weight vector will be the current vector plus the difference between the input vector V
# and the weight vector, multiplied by a learning rate L at time t

# We also identify all the neurons in the SOM that are closer in 2D space than our current radius, and also move them closer to the input vector.
# The difference is that the weight update will be proportional to their 2D distance from the BMU.
# One last thing to note: this proportion of 2D distance isn’t uniform, it’s Gaussian.
# Concretely, this is the equation we’ll use to calculate the influence i
# It = exp(-d²/2σt²)
# where d is the 2D distance and σ is the current radius of our neighbourhood.

def influence(dist, radius):
    return np.exp(-dist**2/(2*radius**2))



In [50]:
# Repeating step n_iterations
for iteration in range(n_iterations):
    # select a random value of the input vector
    t = data[:, np.random.randint(data.shape[1])].flatten()
    # get bmu and coordinates
    bmu, x_min, y_min = find_bmu(t, net)


    r = decay_radius(init_radius, iteration, lambdaa=0.01)
    l = decay_learning_rate(init_learning_rate, iteration, n_iterations)
    
    for x in range(net.shape[0]):
        for y in range(net.shape[1]):
            w = net[x, y, :]
            dist = (x - x_min)**2 + (y - y_min)**2
            if dist < r**2:
                w = w + l*(t-w)*influence(dist, r)
                net[x, y, :] = w



In [51]:
net

array([[[0.72728219, 0.20739531, 0.96658156],
        [0.08983801, 0.27354102, 0.76868801],
        [0.13332453, 0.14685405, 0.52532098],
        [0.71630857, 0.18671411, 0.98179614],
        [0.04658627, 0.78201806, 0.05672755]],

       [[0.85920216, 0.59726002, 0.83412967],
        [0.31580425, 0.39927502, 0.32838809],
        [0.94191924, 0.46237571, 0.48703585],
        [0.96044165, 0.45843714, 0.29769602],
        [0.26317914, 0.01115242, 0.58291375]],

       [[0.79922143, 0.24214217, 0.49953977],
        [0.38035604, 0.49101567, 0.0690822 ],
        [0.73985218, 0.73624268, 0.97836179],
        [0.74515441, 0.72173049, 0.35974995],
        [0.98955662, 0.67363347, 0.66830874]],

       [[0.30856836, 0.31236597, 0.89676327],
        [0.68727367, 0.29189318, 0.30592643],
        [0.95853408, 0.10892847, 0.75139267],
        [0.80442488, 0.76829684, 0.80743147],
        [0.52234568, 0.88208081, 0.27473275]],

       [[0.90393091, 0.3613171 , 0.19527785],
        [0.55961507, 0.395